In [1]:
from PyQt5 import QtCore, QtGui, QtWidgets
from numpy import *
import math
from Optimizermaster.FileHandler import FileHandler
import sys
from gl_widget import GLWidget
from Light import Materials
from OpenGL.GL import *
from OpenGL.arrays import vbo
from OpenGL.GLU import *
from OpenGL.GLUT import *
import matrix
import time
model=[]
class myform(QtWidgets.QWidget):
    
    def __init__(self):
        super(myform,self).__init__()
        #窗口
        self.setObjectName("Form")
        self.resize(1200, 900)
        self.setWindowTitle(QtCore.QCoreApplication.translate("Form", "Form"))
        
        #直接引用自定义opengl类
        self.opengl_widget=GLWidget(self)
        
        #self.btn 打开文件
        self.btn=QtWidgets.QPushButton(self)
        self.btn.setGeometry(QtCore.QRect(710, 40, 75, 23))
        self.btn.setObjectName("pushButton")
        self.btn.setText("打开文件")
        self.btn.clicked.connect(self.getfile)
        
        #self.cb  选择材质
        self.cb=QtWidgets.QComboBox(self)
        self.cb.setGeometry(QtCore.QRect(710,70,100,20))
        self.cb.addItems(['黄铜','青铜','亮青铜','铬','铜','亮铜','金','亮金',
                          '白蜡','银','亮银','祖母绿',
                          '碧玉','黑曜石','珍珠','红宝石','绿宝石','黑塑料','黑橡胶','紫罗兰'])
        self.cb.currentIndexChanged.connect(self.selectionChange)
        #截图按钮
        self.btn_image=QtWidgets.QPushButton(self)
        self.btn_image.setGeometry(QtCore.QRect(710, 100, 75, 23))
        self.btn_image.setObjectName("pushButton_image")
        self.btn_image.setText("截图")
        self.btn_image.clicked.connect(self.getimage)
        #旋转按钮
        self.btn_rotate=QtWidgets.QPushButton(self)
        self.btn_rotate.setGeometry(QtCore.QRect(710, 360, 75, 23))
        self.btn_rotate.setObjectName("pushButton_rotate")
        self.btn_rotate.setText("旋转")
        self.btn_rotate.clicked.connect(self.rote)
        #绕x轴旋转的label
        self.scalex=QtWidgets.QLabel(self)
        self.scalex.setGeometry(QtCore.QRect(795,362,20,20))
        self.scalex.setObjectName('scale_x')
        self.scalex.setText('X: ')
        #绕x轴旋转的输入框
        self.tb_scalex=QtWidgets.QLineEdit(self)
        self.tb_scalex.setGeometry(QtCore.QRect(810,360,45,23))
        self.tb_scalex.setObjectName('tb_scalex')
        self.tb_scalex.setText('0')
        #绕y轴旋转的label
        self.scaley=QtWidgets.QLabel(self)
        self.scaley.setGeometry(QtCore.QRect(860,362,20,20))
        self.scaley.setObjectName('scale_y')
        self.scaley.setText('Y: ')
        #绕y轴旋转的输入框
        self.tb_scaley=QtWidgets.QLineEdit(self)
        self.tb_scaley.setGeometry(QtCore.QRect(875,360,45,23))
        self.tb_scaley.setObjectName('tb_scaley')
        self.tb_scaley.setText('0')
        #绕z轴旋转的label
        self.scalez=QtWidgets.QLabel(self)
        self.scalez.setGeometry(QtCore.QRect(920,362,20,20))
        self.scalez.setObjectName('scale_z')
        self.scalez.setText('Z: ')
        #绕z轴旋转的输入框
        self.tb_scalez=QtWidgets.QLineEdit(self)
        self.tb_scalez.setGeometry(QtCore.QRect(935,360,45,23))
        self.tb_scalez.setObjectName('tb_scalez')
        self.tb_scalez.setText('0')
        #平移按钮
        self.btn_rotate=QtWidgets.QPushButton(self)
        self.btn_rotate.setGeometry(QtCore.QRect(710, 390, 75, 23))
        self.btn_rotate.setObjectName("pushButton_translate")
        self.btn_rotate.setText("平移")
        self.btn_rotate.clicked.connect(self.translate)
        #x方向平移的label
        self.translatex=QtWidgets.QLabel(self)
        self.translatex.setGeometry(QtCore.QRect(795,392,20,20))
        self.translatex.setObjectName('translate_x')
        self.translatex.setText('X: ')
        #x方向平移的输入框
        self.tb_translatex=QtWidgets.QLineEdit(self)
        self.tb_translatex.setGeometry(QtCore.QRect(810,390,45,23))
        self.tb_translatex.setObjectName('tb_translatex')
        self.tb_translatex.setText('0')
        #y方向平移的label
        self.translatey=QtWidgets.QLabel(self)
        self.translatey.setGeometry(QtCore.QRect(860,392,20,20))
        self.translatey.setObjectName('translate_y')
        self.translatey.setText('Y: ')
        #y方向平移的输入框
        self.tb_translatey=QtWidgets.QLineEdit(self)
        self.tb_translatey.setGeometry(QtCore.QRect(875,390,45,23))
        self.tb_translatey.setObjectName('tb_translatey')
        self.tb_translatey.setText('0')
        #z方向平移的label
        self.translatez=QtWidgets.QLabel(self)
        self.translatez.setGeometry(QtCore.QRect(920,392,20,20))
        self.translatez.setObjectName('translate_z')
        self.translatez.setText('Z: ')
        #z方向平移的输入框
        self.tb_translatez=QtWidgets.QLineEdit(self)
        self.tb_translatez.setGeometry(QtCore.QRect(935,390,45,23))
        self.tb_translatez.setObjectName('tb_translatez')
        self.tb_translatez.setText('0')
        
        
        #旋转相机按钮
        self.btn_rotate=QtWidgets.QPushButton(self)
        self.btn_rotate.setGeometry(QtCore.QRect(710, 430, 75, 23))
        self.btn_rotate.setObjectName("pushButton_rotate_camera")
        self.btn_rotate.setText("旋转相机")
        self.btn_rotate.clicked.connect(self.rotate_camera)
        self.isrotatecamera=False
        self.timer=QtCore.QTimer(self)
        self.timer.timeout.connect(self.ro)
        #旋转相机定时器启动时间输入框，数字越大，速度越慢
        self.tb_rotatecamera=QtWidgets.QLineEdit(self)
        self.tb_rotatecamera.setGeometry(QtCore.QRect(800,430,45,23))
        self.tb_rotatecamera.setObjectName('tb_rotatecamera')
        self.tb_rotatecamera.setText('50')
        #为两个工字钢定制的旋转按钮
        self.btn_rotate=QtWidgets.QPushButton(self)
        self.btn_rotate.setGeometry(QtCore.QRect(710, 460, 75, 23))
        self.btn_rotate.setObjectName("pushButton_default")
        self.btn_rotate.setText("归位")
        self.btn_rotate.clicked.connect(self.default)
        #工字钢旋转角度输入框，数字越小，速度越慢
        self.tb_default=QtWidgets.QLineEdit(self)
        self.tb_default.setGeometry(QtCore.QRect(800,460,45,23))
        self.tb_default.setObjectName('tb_default')
        self.tb_default.setText('5')
        self.isdefault=False
        self.timerdefault=QtCore.QTimer(self)
        self.timerdefault.timeout.connect(self.time_default)
        
        self.qlistwidget=QtWidgets.QListWidget(self)
        self.qlistwidget.setGeometry(QtCore.QRect(720,140,100,200))
        self.qlistwidget.setObjectName('Object Name')
    def getfile(self):
        dlg=QtWidgets.QFileDialog()
        f=dlg.getOpenFileName(self,'Open File','.','STL File (*.stl)')
        file=FileHandler().load_mesh(f[0])#f[0]是路径，f[1]是类型
        data=array(file['mesh'],'f')
        #print(data)
        model.append({'material':Materials[0],'data':data})
        self.qlistwidget.addItem(f[0].split('/')[-1].split('.')[0])
        self.qlistwidget.setCurrentItem(self.qlistwidget.item(0))
        self.opengl_widget.initdata(model)
        #print(data)
        self.opengl_widget.update()
    def selectionChange(self):
        print(self.cb.currentIndex())
        self.opengl_widget.model[self.qlistwidget.currentIndex().row()]['material']=Materials[self.cb.currentIndex()]
        self.opengl_widget.update()
    def getimage(self):
        #glutSwapBuffers()
        self.opengl_widget.geti()
    def rote(self):
        matrix.rotate(model[self.qlistwidget.currentIndex().row()]['data'],int(self.tb_scalex.text()),x=1)
        matrix.rotate(model[self.qlistwidget.currentIndex().row()]['data'],int(self.tb_scaley.text()),y=1)
        matrix.rotate(model[self.qlistwidget.currentIndex().row()]['data'],int(self.tb_scalez.text()),z=1)
        self.opengl_widget.update()
    def translate(self):
        matrix.translate(model[self.qlistwidget.currentIndex().row()]['data'],int(self.tb_translatex.text()),int(self.tb_translatey.text()),int(self.tb_translatez.text()))
        self.opengl_widget.update()
    def rotate_camera(self):
        if self.isrotatecamera==False:
            self.isrotatecamera=True
            self.timer.start(int(self.tb_rotatecamera.text()))
        else:
            self.isrotatecamera=False
            self.timer.stop()
    def ro(self):
        self.opengl_widget.camera.Yaw(5)
        self.opengl_widget.light.lightpos=[-self.opengl_widget.camera.forwardDir.X,-self.opengl_widget.camera.forwardDir.Y,-self.opengl_widget.camera.forwardDir.Z,0]
        self.opengl_widget.update()
    def default(self):
        self.timerdefault.start(80)
        self.rotate=0
    def time_default(self):
        angle=int(self.tb_default.text())
        matrix.rotate(model[1]['data'],angle,x=1)
        matrix.rotate(model[2]['data'],-angle,x=1)
        self.opengl_widget.update()
        self.rotate=self.rotate+angle
        if self.rotate==90:
            self.timerdefault.stop()
app=QtWidgets.QApplication(sys.argv)
mywindow=myform()
mywindow.show()
app.exec_()
sys.exit()

b'\xfcc\x0f\x00'
(1008636,)


SystemExit: 

d:\Users\PC\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
